#### 🎬 PROYECTO DE WEB SCRAPING (SELENIUM - PANDAS - PLOTLY) 🚀  

##### 📊 Análisis y Visualización de Datos de las **250 Mejores Películas** según IMDb  
📌 **Descripción:**  
Este proyecto utiliza **Selenium** para realizar **Web Scraping** en [IMDb Top 250](https://www.imdb.com/chart/top/) y extraer 
información clave de las películas más destacadas.  

📌 **Objetivo:**  
✔ Extraer y procesar datos de IMDb 📂  
✔ Analizar tendencias en películas top 🎥  
✔ Visualizar insights con gráficos 📊  

📌 **Tecnologías utilizadas:**  
🔹 **Selenium** → Automatización del scraping 🕵️  
🔹 **Pandas** → Manejo y transformación de datos 🗂  
🔹 **Plotly** → Visualización de patrones y tendencias 📈

######  CONTINUAR HOY 21/03/2025

 ### 📌 **Redes Profesionales y Portafolio:**  
🔗 [LinkedIn](https://www.linkedin.com/in/brayan-rafael-neciosup-bola%C3%B1os-407a59246/)  
🔗 [GitHub](https://github.com/BrayanR03)  
🔗 [Portafolio de Proyectos](https://bryanneciosup626.wixsite.com/brayandataanalitics)  

### WEB SCRAPING 👇📊

In [ ]:
# IMPORTAMOS LIBRERIAS
from selenium import webdriver # Driver para utlizar con selenium
from selenium.webdriver.common.by import By # Libreria para porder acceder a traves de IDs, class y XPATH
from selenium.webdriver.support.ui import WebDriverWait # Brinda un tiempo de espera hasta que todos los elementos de la web esten listos.
import pandas as pd # Librería para la manipulación de la información
import re # Librería para trabajar con expresiones regulares
from selenium.webdriver.support import expected_conditions as EC # Permite verificar si un o varios elementos estan listos para interactuar
from selenium.webdriver.chrome.options import Options # Opciones para el navegador


In [ ]:

web_url = "https://www.imdb.com/chart/top/" # URL DE LA WEB
path = "webscraping/chromedriver-win64"

# ABRIMOS EL DRIVER
driver = webdriver.Chrome() # Permite abrir la pagina en primer plano
driver.get(web_url) # OBTENEMOS LA INFORMACIÓN DE LA WEB

wait = WebDriverWait(driver, 20) # Brinda un tiempo de espera hasta que todos los elementos de la web esten listos.

try:
    
    div_movies = wait.until(EC.presence_of_element_located((By.XPATH,'//div[@data-testid="chart-layout-main-column"]')))
    ul_movies = WebDriverWait(div_movies,10).until(
        EC.presence_of_element_located((By.XPATH,'//ul[@role="presentation"]'))
    )
    
    # Extraemos el título de la película
    title_movie = [i.text for i in WebDriverWait(ul_movies,15).until(EC.presence_of_all_elements_located((By.TAG_NAME,'h3')))]
    
    # Extraemos los detalles de la película (Año de estreno, duración (horas y minutos) y clasificación)
    details_movie = [i.text for i in  WebDriverWait(ul_movies,15).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="sc-f30335b4-6 kGhnhC cli-title-metadata"]')))]
    
    # Extraemos el promedio de estrellas que tiene la película
    star_rating_movie= [i.text for i in WebDriverWait(ul_movies,15).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'ipc-rating-star--rating')))]
    
    # Extraemos la cantidad de votos que tiene la película
    star_vote_count_movie = [i.text for i in WebDriverWait(ul_movies,15).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'ipc-rating-star--voteCount')))]
    print("WEB SCRAPING REALIZADO")
    # IMPORTANTE: Debemos buscar especificamente con que elementos interactuar, para evitar la búsqueda de todos los elementos en la página.
    driver.quit() # Cerramos el proceso de selenium con el navegador
except Exception as e:
    print(f"error al acceder al main: {e}")
    # driver.quit() # Cerramos el proceso de selenium con el navegador

WEB SCRAPING REALIZADO


### DATA WRANGLING (MANIPULACIÓN DE DATOS) 👇📊

In [162]:
# En esta sección realizaremos la manipulación de datos con la información de las películas.

# Almacenamos en un diccionario la información, para una rápida transformación a un DATAFRAME
diccionario_movie = {
    "Title Movie": title_movie,
    "Detail Movie": details_movie,
    "Star Rating Movie": star_rating_movie,
    "Star Vote Count Movie": star_vote_count_movie
}
df_movies = pd.DataFrame(diccionario_movie)
df_movies.head()

,Title Movie,Detail Movie,Star Rating Movie,Star Vote Count Movie
0,1. Sueño de fuga,1994\n2h 22m\nB,9.3,(3 M)
1,2. El padrino,1972\n2h 55m\nC,9.2,(2.1 M)
2,3. Batman: El caballero de la noche,2008\n2h 32m\nB,9.0,(3 M)
3,4. El padrino (parte II),1974\n3h 22m\nC,9.0,(1.4 M)
4,5. 12 hombres en pugna,1957\n1h 36m\nB,9.0,(918 k)


In [163]:
# Separamos la columna Detail Movie en 3 columnas (Year Movie - Duration Movie - Clasification Movie)
df_movies[["Year Movie","Duration Movie","Clasification Movie"]] = df_movies["Detail Movie"].str.split('\n',expand=True)
# Eliminamos la columna Detail Movie
df_movies.drop("Detail Movie",inplace=True,axis=1)
df_movies.head()

,Title Movie,Star Rating Movie,Star Vote Count Movie,Year Movie,Duration Movie,Clasification Movie
0,1. Sueño de fuga,9.3,(3 M),1994,2h 22m,B
1,2. El padrino,9.2,(2.1 M),1972,2h 55m,C
2,3. Batman: El caballero de la noche,9.0,(3 M),2008,2h 32m,B
3,4. El padrino (parte II),9.0,(1.4 M),1974,3h 22m,C
4,5. 12 hombres en pugna,9.0,(918 k),1957,1h 36m,B


In [165]:
# Tranformamos la columna Title Movie (Eliminamos el número, puntos, espacios que aparecía en la página)
# Función para extraer el nombre de la película
def extraer_nombre(nombre_pelicula):
    match = re.search(r'^\s*\d+\.\s*(.*)',str(nombre_pelicula)) # Busca y extrae el nombre, con los prefijos (1. ) 
    return match.group(1).strip() if match else nombre_pelicula
df_movies["Title Movie"] = df_movies["Title Movie"].apply(extraer_nombre)
df_movies.head()

,Title Movie,Star Rating Movie,Star Vote Count Movie,Year Movie,Duration Movie,Clasification Movie
0,Sueño de fuga,9.3,(3 M),1994,2h 22m,B
1,El padrino,9.2,(2.1 M),1972,2h 55m,C
2,Batman: El caballero de la noche,9.0,(3 M),2008,2h 32m,B
3,El padrino (parte II),9.0,(1.4 M),1974,3h 22m,C
4,12 hombres en pugna,9.0,(918 k),1957,1h 36m,B


In [166]:
# Transformamos la columna Star Vote Count Movie (Millones)
# Extraemos las cifras omitiendo "()", "M-k" y los convertimos a float para su análisis posterior.
# Las cifras que empiecen "0.", las transformamos para que se estandaricen a la columna en Millones
df_movies["Star Vote Count Movie"] = df_movies["Star Vote Count Movie"].apply(lambda x : float(x[2:-3:])/1000 if x[-2:-1:]=='k' else float(x[2:-3:]))
# Cambiamos el nombre de la columna y establecemos que los valores con en Millones o próximos.
df_movies = df_movies.rename({"Star Vote Count Movie":"Star Vote Count Movie (M)"},axis=1)
df_movies.head()

,Title Movie,Star Rating Movie,Star Vote Count Movie (M),Year Movie,Duration Movie,Clasification Movie
0,Sueño de fuga,9.3,3.000,1994,2h 22m,B
1,El padrino,9.2,2.100,1972,2h 55m,C
2,Batman: El caballero de la noche,9.0,3.000,2008,2h 32m,B
3,El padrino (parte II),9.0,1.400,1974,3h 22m,C
4,12 hombres en pugna,9.0,0.918,1957,1h 36m,B


In [167]:
# Dividimos la columna Duration Movie en: Duration Hours - Duration Minutes

# Funciones para extraer horas y minutos
def extraer_horas(duracion):
    match = re.search(r"(\d+)h", str(duracion))  # Busca el número que tiene de subfijo (h)
    return int(match.group(1)) if match else 0   # Retornará 0, si no hay horas (Por ejemplo, si el dato es: 45m, 25m)

def extraer_minutos(duracion):
    match = re.search(r"(\d+)m", str(duracion))  # Busca el número que tiene de subfijo (m)
    return int(match.group(1)) if match else 0   # Retornará 0, si no hay minutos (Por ejemplo, si el dato es: 2h, 3h)

# Creamos la columna Duration Hours
df_movies["Duration Hours"] = df_movies["Duration Movie"].apply(extraer_horas)

# Creamos la columna Duration Minutes
df_movies["Duration Minutes"] = df_movies["Duration Movie"].apply(extraer_minutos)
# Eliminamos la columna Duration Movies
df_movies.drop("Duration Movie",axis=1,inplace=True)
df_movies.head()

,Title Movie,Star Rating Movie,Star Vote Count Movie (M),Year Movie,Clasification Movie,Duration Hours,Duration Minutes
0,Sueño de fuga,9.3,3.000,1994,B,2,22
1,El padrino,9.2,2.100,1972,C,2,55
2,Batman: El caballero de la noche,9.0,3.000,2008,B,2,32
3,El padrino (parte II),9.0,1.400,1974,C,3,22
4,12 hombres en pugna,9.0,0.918,1957,B,1,36


In [168]:
# Verificamos existencia de datos nulos
df_movies.isnull().sum()

Title Movie                  0
Star Rating Movie            0
Star Vote Count Movie (M)    0
Year Movie                   0
Clasification Movie          3
Duration Hours               0
Duration Minutes             0
dtype: int64

In [174]:
# Reemplazamos valores nulos de la columna (Clasification Movie) por "S/C"
df_movies["Clasification Movie"].fillna("S/C",inplace=True)
print("Valores reemplazados")

Valores reemplazados


In [176]:
# Verificamos que los valores nulos han sido reemplazados
df_movies.isnull().sum()

Title Movie                  0
Star Rating Movie            0
Star Vote Count Movie (M)    0
Year Movie                   0
Clasification Movie          0
Duration Hours               0
Duration Minutes             0
dtype: int64

In [177]:
# Exportamos nuestro dataset limpio y transformado a un .CSV
df_movies.to_csv('MoviesTop250IMDB.csv',index=False,sep=',')
print("Se exportó la información")

Se exportó la información


### VISUALIZACIÓN DE DATOS 👇📊

In [ ]:
import plotly.express as px # Librería para crear gráficos interactivos

In [179]:
carpeta = r'../webscraping/'
archivo = 'MoviesTop250IMDB.csv'
dataset_movies = pd.read_csv(carpeta+archivo,sep=',')
dataset_movies.head()


,Title Movie,Star Rating Movie,Star Vote Count Movie (M),Year Movie,Clasification Movie,Duration Hours,Duration Minutes
0,Sueño de fuga,9.3,3.000,1994,B,2,22
1,El padrino,9.2,2.100,1972,C,2,55
2,Batman: El caballero de la noche,9.0,3.000,2008,B,2,32
3,El padrino (parte II),9.0,1.400,1974,C,3,22
4,12 hombres en pugna,9.0,0.918,1957,B,1,36


In [209]:
# 1 Comparación de Calificaciones Promedio por Año

df_promedio = dataset_movies.groupby(by="Year Movie",as_index=False,observed=True)["Star Rating Movie"].mean()
line_plot = px.line(data_frame=df_promedio,x="Year Movie",y="Star Rating Movie",markers=True)
line_plot.update_layout(
    title = "Evolución Promedio de Star Rating Movie",
    title_x = 0.5,
    xaxis_title = 'Estrellas por Película',
    yaxis_title = 'Año',
    hoverlabel = dict(
        bgcolor = "white",
        font_color = "black"
    )
)
line_plot.update_traces(
    hovertemplate = "Año:%{x}<br>Clasificación Estrella:%{y}<extra></extra>",
    line = dict(
        color = "red"
    )
)
line_plot.show()

In [201]:
# 2: Top 5 Películas con mayor duración
top_10_peliculas = dataset_movies.sort_values(by="Total Duration Minutes",ascending=False).head(10)
hist_plot = px.histogram(data_frame=top_10_peliculas,x="Title Movie",y="Duration Minutes",color="Title Movie")
hist_plot.update_layout(
    title = "TOP 10 PELÍCULAS CON MAYOR DURACIÓN",
    title_x = 0.5,
    showlegend = False,
    xaxis_title = "Título Película",
    yaxis_title = "Duración (Minutos)",
    hoverlabel = dict(
        bgcolor = "green"
    )
)
hist_plot.update_traces(
    hovertemplate = "Duración (Minutos): %{y}<br>Título:%{x}<extra></extra>"
)
hist_plot.show()


In [181]:
# 3: Relación entre Votos y Calificación de Películas
scatter_plot = px.scatter(data_frame=dataset_movies,x="Star Rating Movie",y="Star Vote Count Movie (M)",size="Star Vote Count Movie (M)",)
scatter_plot.update_layout(
    title = "Star Rating vs Star Vote",
    xaxis_title = "Calificación Estrellas",
    yaxis_title = "Cantidad Votos (Millones)",
    hoverlabel = dict(
        bgcolor = "green"
    )
)
scatter_plot.update_traces(
    hovertemplate = "Cantidad Votos:%{y}<br>Calificación Estrellas:%{x}<extra></extra>"
)
scatter_plot.show()

In [182]:
# 4: Clasificación de Películas más Común
df_clasificacion_movie = dataset_movies.groupby(by="Clasification Movie",as_index=False,observed=True).size()
# df_clasificacion_movie.head()
bar_plot = px.bar(data_frame=df_clasificacion_movie,x="Clasification Movie",y="size",color="Clasification Movie")
bar_plot.update_layout(
    title = "Cantidad de Películas por Clasificación",
    title_x = 0.5,
    showlegend = False,
    hoverlabel = dict(
        bgcolor = 'blue',
        font_color = "white"
    ),
    xaxis_title = "Clasificación",
    yaxis_title = "Cantidad"
    
)
bar_plot.update_traces(
    hovertemplate = "Cantidad:%{y}<br>Clasificación:%{x}<extra></extra>"
)
bar_plot.show()

In [194]:
# 5: Comparación de Duración Media por Clasificación
dataset_movies["Total Duration Minutes"] = (dataset_movies["Duration Hours"]*60)+dataset_movies["Duration Minutes"]
box_plot = px.box(data_frame=dataset_movies,x="Clasification Movie",y="Total Duration Minutes",color="Clasification Movie")
box_plot.update_layout(
    showlegend = False,
    title = "Promedio Duración Películas",
    xaxis_title = "Clasificación Película",
    yaxis_title = "Duración Total (Minutos)",
    hoverlabel = dict(
        bgcolor = "white",
        font_color = 'black'
    )
)
box_plot.show()